In [ ]:
import numpy as np
import pandas as pd
import spatialdata as sd
from geopandas import GeoDataFrame
from shapely import Polygon
from spatialdata.models import ShapesModel
from spatialdata.transformations import Identity
import os
import spatialdata_plot
import spatialdata_io
from napari_spatialdata import Interactive

In [ ]:
Fast1 = sd.read_zarr("10W_FAST_1_9_16um.zarr")
print(any(Fast1.tables['square_016um'].var_names.duplicated()))

In [ ]:
viewer = Interactive(Fast1)
viewer.run()

In [ ]:
Fast1.shapes["mPFC"].geometry.iloc[0]

In [ ]:
shape_names = list(Fast1.shapes.keys())
print(shape_names)

In [ ]:
from spatialdata import polygon_query

sdata = Fast1
shape_to_query = 'mPFC'
target_coordinate_system = '10W_FAST_1_9'


polygon = sdata.shapes[shape_to_query].geometry.iloc[0]
query_result = polygon_query(
    sdata,
    polygon=polygon,
    target_coordinate_system= target_coordinate_system
)

cells_inside = query_result['square_016um']
print(f"in '{shape_to_query}' cells: {len(cells_inside)}")

sdata['square_016um'].obs['annotation'] = 'unassigned'
in_mPFC = cells_inside.obs.index

sdata['square_016um'].obs.loc[in_mPFC, 'annotation'] = shape_to_query

print("\n sum:")
print(sdata['square_016um'].obs['annotation'].value_counts())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 10))
ax = plt.gca()

sdata.pl.render_images(
    '10W_FAST_1_9_lowres_image'
).pl.render_shapes(
    '10W_FAST_1_9_square_016um',
    color='annotation'
).pl.show(
    coordinate_systems='10W_FAST_1_9',
    ax=ax
)

plt.show()

In [ ]:
from spatialdata_io.experimental import to_legacy_anndata

Fast1_adata = to_legacy_anndata(
    sdata=sdata,
    coordinate_system='10W_FAST_1_9',
    table_name='square_016um',
    include_images=True
)
print(Fast1_adata)

In [ ]:
mask = Fast1_adata.obs['annotation'] == 'mPFC'
Fast1_adata = Fast1_adata[mask].copy()

print(mask.sum(), "spots kept")

In [ ]:

Fast1_adata.write_h5ad("Fast1_16um_mPFC_adata.h5ad")